# AWS 서비스를 사용해 자동 자막 만들기

### 사용할 AWS 기능
- AWS transcribe
- AWS S3
- AWS Lambda
- AWS IAM
- AWS CLI

### 1. python과 AWS 서비스 연동

transcribe 서비스는 기본적으로 AWS 콘솔에 접속해 사용할 수 있다. AWS는 요금정책이 철저하고 보안이 강력해서 해당 서비스를 개인 pc 환경에서 구동하려면 AWS CLI(AWS Command Libe Interface)를 설치해야 한다. 해당 링크를 통해 [AWS CLI를 설치](https://docs.aws.amazon.com/ko_kr/cli/latest/userguide/cli-chap-install.html)하고 잘 설치되었는지 확인한다.

In [4]:
!aws --version

aws-cli/2.0.14 Python/3.7.7 Windows/10 botocore/2.0.0dev18


AWS IAM으로 권한 설정을 마친 사용자를 등록한다. 해당 계정은 최소한의 권한만을 허용하기 위해 AmazonS3FullAccess와 AmazonTranscribeFullAccess 권한을 부여했다.

`aws configure`를 사용해 사용자를 등록한다.

환경 설정 완료 후 python에서 코드를 구동하기 전에 [boto3을 설치](https://github.com/boto/boto3)한다. boto3 설치,IAM권한 설정, CLI 설정이 끝났다면 연결 확인을 위해 개인 pc의 python환경에서 AWS SDK를 사용하는 예제를 실행한다.

In [6]:
import boto3

s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

psjtranscribebucket


### 2. python으로 S3 제어하기
AWS가 제공하는 transcribe 서비스는 기본적으로 다음과 같은 순서로 진행된다.

1. S3에 transcribe를 원하는 파일(영상 or 음성)을 업로드한다.
2. transcribe에서 job을 생성한다.
3. S3에 업로드된 파일들 중 transcribe를 적용할 파일의 uri를 입력한다.
4. 결과 json파일을 다운받을 수 있는 외부링크로 받는다.

최종 목표는 python으로 영상파일을 넘기면 자동 자막이 생성된 영상을 받는 것이지만, 요금이 낭비되는 것을 방지하기 위해 일단 음성 파일을 번역하는 process를 만들고자 한다.

자동 자막화 프로그램에서는 S3를 통해 파일을 업로드하고, 업로드한 url을 확인한 뒤 삭제하는 작업을 수행한다. 코드는 [Boto3 docs](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-examples.html)의 S3 예제를 참고해 작성했다. 아래의 코드는 <b>S3에 파일을 업로드</b>하는 코드이다.

In [ ]:
# S3에 파일 업로드하기
import boto3

# S3 클라이언트 생성
s3 = boto3.client('s3')

# 업로드할 파일의 이름
filename = 'transcribe-sample.mp3'
# filename = 'audio/transcribe-sample.mp3'처럼 폴더 내의 파일을 업로드하면 audio폴더도 업로드된다.

# 파일을 담을 S3 버킷 이름
bucket_name = 'psjtranscribebucket'

s3.upload_file(filename, bucket_name, filename)

transcribe 서비스는 위에서 언급한 바와 같이, S3에 업로드된 파일의 uri를 필요로 한다.따라서 python을 사용해 <b>uri를 가져오는 예제</b>를 다음과 같이 작성한다.

In [1]:
# S3에 업로드된 파일의 uri 가져오기
import boto3

# s3 클라이언트 생성
s3 = boto3.client('s3')

# 버켓이름
bucket_name = 'psjtranscribebucket'

# 파일이름
filename = 'transcribe-sample.mp3'

location = s3.get_bucket_location(Bucket=bucket_name)['LocationConstraint']
url = "https://s3-%s.amazonaws.com/%s/%s" % (location, bucket_name, filename)
print(url)

https://s3-ap-northeast-2.amazonaws.com/psjtranscribebucket/transcribe-sample.mp3


uri 확인이 끝났다면 업로드된 파일을 삭제해야 한다. 코드는 다음과 같다.

In [8]:
# S3에 업로드된 파일 삭제하기
import boto3

# s3 클라이언트 생성
s3 = boto3.client('s3')

# 삭제할 파일이 들어있는 버켓 이름
bucket_name = 'psjtranscribebucket'

# 삭제할 파일 이름 지정
filename = 'transcribe-sample.mp3'

# 파일 삭제 후 확인
response = s3.delete_object(Bucket=bucket_name, Key=filename)
print(response)

{'ResponseMetadata': {'RequestId': '8B56EEDF53D39222', 'HostId': 'q61qVCFFVDdt6pKe6/S2GBz0lFRxD4dFxYF//DaPrs+ZYY0NxMF6w3cFMTPiFSLlWZAVQpuN6pw=', 'HTTPStatusCode': 204, 'HTTPHeaders': {'x-amz-id-2': 'q61qVCFFVDdt6pKe6/S2GBz0lFRxD4dFxYF//DaPrs+ZYY0NxMF6w3cFMTPiFSLlWZAVQpuN6pw=', 'x-amz-request-id': '8B56EEDF53D39222', 'date': 'Wed, 27 May 2020 12:49:05 GMT', 'server': 'AmazonS3'}, 'RetryAttempts': 0}}


### 3. python으로 transcribe 제어하기

S3에 업로드된 파일을 transcribe로 번역한다. 번역의 결과물은 json파일을 다운받을 수 있는 URL로 받는다.

In [4]:
from __future__ import print_function
import time
import boto3

transcribe = boto3.client('transcribe')
job_name = "sdk-test"
job_uri = url
#aws S3에 업로드한 파일의 url 입력
transcribe.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri':job_uri},
    MediaFormat='mp3',
    LanguageCode='en-US'
)
while True:
    status = transcribe.get_transcription_job(TranscriptionJobName=job_name)
    if status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    print("Not ready yet...")
    time.sleep(5)
print(status)

Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
Not ready yet...
{'TranscriptionJob': {'TranscriptionJobName': 'sdk-test', 'TranscriptionJobStatus': 'COMPLETED', 'LanguageCode': 'en-US', 'MediaSampleRateHertz': 44100, 'MediaFormat': 'mp3', 'Media': {'MediaFileUri': 'https://psjtranscribebucket.s3.ap-northeast-2.amazonaws.com/transcribe-sample.mp3'}, 'Transcript': {'TranscriptFileUri': 'https://s3.ap-northeast-2.amazonaws.com/aws-transcribe-ap-northeast-2-prod/324820801113/sdk-test/eb3aceee-891f-412c-a6fc-d8b3fd30977b/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGQaDmFwLW5vcnRoZWFzdC0yIkcwRQIhAKpoMAL%2FWb%2B6iuL2tAED6Up6%2FVPQg8pu5Y%2B1ZWnisJnRAiArYzRmoNgVcdEXvFusyz1tNcYotE6i74wMcPjm92PS%2ByrHAwi9%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAEaDDQzNTc2MzIxMDc3NiIMAj4hN13Z0lFBA8tKKpsD9A57udiEogmv1%2FIrqj3ViympIMQCJxW9roXJzU%2F5tZCwN%2BSfpp5d3Ab7g%2Bh099Lt

In [6]:
import webbrowser

url = status["TranscriptionJob"]["Transcript"]["TranscriptFileUri"]

# 결과 json파일을 다운받는 url 오픈
webbrowser.open(url)

https://s3.ap-northeast-2.amazonaws.com/aws-transcribe-ap-northeast-2-prod/324820801113/sdk-test/eb3aceee-891f-412c-a6fc-d8b3fd30977b/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEGQaDmFwLW5vcnRoZWFzdC0yIkcwRQIhAKpoMAL%2FWb%2B6iuL2tAED6Up6%2FVPQg8pu5Y%2B1ZWnisJnRAiArYzRmoNgVcdEXvFusyz1tNcYotE6i74wMcPjm92PS%2ByrHAwi9%2F%2F%2F%2F%2F%2F%2F%2F%2F%2F8BEAEaDDQzNTc2MzIxMDc3NiIMAj4hN13Z0lFBA8tKKpsD9A57udiEogmv1%2FIrqj3ViympIMQCJxW9roXJzU%2F5tZCwN%2BSfpp5d3Ab7g%2Bh099LtVV1Cd33jJJ8voPp3x697wmkbXT2cWiaL4PTKDKc7axy6g8aEc6WkL234wMeUvOXiEcreGgQs%2Fad7fqezAeujOX8Ark%2B3tvvs4YfB30H%2FzBVd28KVHeQ7syygEq7ieyxmjumx3wvyUBCRPffwLh3X0f5i5V64xpXml1NRP6N94zBHmplJOEX%2BsU%2BwXkCROocgoXU6xwIpSeSNRK9k1S2fKuJOOfhTSqBtIiM2xJ2dCZrTnPk1ZQFs51O0Li7E7YrGDh1LkRJp2sX0g9PKFHvgLs3SoACr1sPk32SR6NBIsaYfS3Z8lJZJQRngVv23e8GhPWgx4%2FUqILsEsYEI6W%2FEaYbqg8B7y7Tm10Y4UBQl8l4MtJTK5LUujkakO%2FjXYtS45CK9LkUG%2BBTknO3PKO5ueP5MlnoJgjM%2BHmEpDgaA2mI6wzJJml1TtrOgKuN6srcJt3wbeCjBfqCBIeuoyrqtb9QUJ%2BmHaD0y%2F2HOMParufYFOusBzBTqrIgX

True

transcribe 작업이 끝난 후 job을 삭제해야 한다. AWS는 동일한 이름의 job을 복수 개 만들 수 없게 설정해뒀기 때문에, 코드 재사용시 충돌을 막기 위해 job을 삭제한다. job을 삭제하는 코드는 아래와 같다.

In [7]:
import boto3

job_name = "sdk-test"

transcribe = boto3.client('transcribe')
transcribe.delete_transcription_job(TranscriptionJobName=job_name)

{'ResponseMetadata': {'RequestId': '91b4e13d-30c6-4281-99a1-5009b901f8f8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 27 May 2020 12:39:26 GMT',
   'x-amzn-requestid': '91b4e13d-30c6-4281-99a1-5009b901f8f8',
   'content-length': '0',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

### 4. AWS Lambda로 서비스 묶어주기

AWS Lambda는 <b>이벤트에 응답하여 코드를 실행하고 자동으로 기본 컴퓨팅 리소스를 관리하는 서버 없는 컴퓨팅 서비스</b>이다. 본 프로젝트에서는 AWS내의 복수개의 서비스를 하나로 묶어서 실행시키기 위해 사용한다. 미리 사용할 Lambda 함수를 정의하면 python에서 한 코드의 실행만으로 Lambda에 정의된 대로 다음의 일련의 작업이 수행된다.